<a href="https://colab.research.google.com/github/guebin/DL2024/blob/main/posts/08wk-1-2.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상 

In [445]:
#{{<video https://youtu.be/playlist?list=PLQqh36zP38-xfh-AQQI0B_GONOjgj9DCi&si=uWImDc1bYBoNqCB_ >}}

# 2. Imports 

In [448]:
import torch
import pandas as pd

# 3. 예비학습: optimizer 사용 고급 

# 4. 추천시스템의 학습전략

ref: <https://namu.wiki/w/나는%20SOLO>

## A. Data: 나는 SOLO

`-` Data 

In [28]:
#df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2024/main/posts/solo.csv',index_col=0)
df_view = pd.read_csv('solo.csv',index_col=0)
df_view

,영식(IN),영철(IN),영호(IS),광수(IS),상철(EN),영수(EN),규빈(ES),다호(ES)
옥순(IN),4.5,4.0,3.2,NaN,2.1,NaN,1.3,1.2
영자(IN),NaN,4.3,3.5,3.4,2.3,2.6,NaN,1.8
정숙(IS),3.8,NaN,4.3,4.6,1.5,1.0,NaN,2.5
영숙(IS),3.9,3.1,NaN,4.1,1.2,NaN,2.1,2.3
순자(EN),NaN,2.9,1.2,1.9,NaN,4.5,3.9,3.2
현숙(EN),2.1,2.4,NaN,1.5,4.5,4.5,3.9,NaN
서연(ES),1.8,1.0,2.3,NaN,NaN,3.9,4.9,4.8
보람(ES),NaN,1.9,2.2,3.1,3.7,4.2,4.2,NaN
하니(I),4.9,4.7,4.9,4.9,4.2,4.1,4.3,3.9


`-` 데이터를 이해할 때 필요한 가정들 -- 제가 마음대로 설정했어요.. 

- 궁합이 잘맞으면 5점, 잘 안맞으면 0점 이다. 
- ((옥순,영자,정숙),(영식,영철,영호))은 MBTI가 I로 시작하고 ((영숙,순자,현숙),(광수,상철,영수))는 MBTI가 E로 시작한다고 설정하자.
- (옥순,영자,정숙)은 (영식,영철,영호)와 성격이 잘 맞고 (영숙,순자,현숙)은 (광수,상철,영수)와 성격이 잘맞음. 
- 정숙은 대체적으로 모든 사람들이랑 궁합이 잘 맞는 편인것 같다. 
- 현숙은 전체적으로 모든 사람들이랑 궁합이 잘 안맞는 편인것 같다. (눈이 높아보인다)

## B. Fit / Predict

`-` 목표: NaN을 추정 

`-` 수동추론: 

- (옥순,영호)이 만난다면? $\to$ 둘다 I성향이니까 잘 맞지 않을까? $\to$ 4.0 정도? 
- (정숙,영식)조합은? $\to$ 둘다 I성향이니까 잘 맞지 않을까? + 정숙은 다 잘맞던데..? $\to$ 4.8 정도? 
- (현숙,영식)조합은? $\to$ 현숙은 E성향인데 영식은 I성향이므로 잘 안맞을 것임 + 현숙은 원래 좀 눈이 높음 $\to$ 0.25 정도? 

`-` 좀 더 체계적인 추론 전략: 사람들이 가지고 있는 성향들을 E가 클수록 양수, I가 클수록 음수로 표현하자. 

- 옥순의 성향 = -1.0
- 영식의 성향 = +1.1
- 현숙의 성향 = (I성향,E성향) = (0.0, 1.5) 

> 사람에 따라 E,I 둘다 높게 설정할수도있고, 둘다 낮게 설정할 수도 있다. 

**(1)** 옥순과 영식의 궁합 $\approx$ 옥순의IE성향$\times$영식의IE성향 $+$ 옥순의NS성향$\times$영식의NS성향 // 적합

In [29]:
옥순성향 = torch.tensor([1.9,0.0]).reshape(2,1) 
영식성향 = torch.tensor([2.0,0.1]).reshape(2,1) 
#(옥순성향*영식성향).sum().item() # 옥순과 영식의 궁합: a ∘ b 로 내적구함 
(옥순성향.T @ 영식성향).item() # 옥순과 영식의 궁합: a.T @ b 로 내적구함 

3.799999952316284

**(2)** 현숙과 영식의 궁합 $\approx$ 현숙의IE성향$\times$영식의IE성향 $+$ 현숙의NS성향$\times$영식의NS성향 // 예측

In [30]:
현숙성향 = torch.tensor([0.0,1.5]).reshape(2,1)
#(현숙성향*영식성향).sum().item() # 현숙과 영식의 궁합: a ∘ b 로 내적구함 
(현숙성향.T @ 영식성향).item() # 현숙과 영식의 궁합: a.T @ b 로 내적구함 

0.15000000596046448

- 그럴싸함. 

`-` 전체 사용자의 설정값 

In [31]:
옥순성향 = a1 = torch.tensor([1.9,0.0]).reshape(2,1)
영자성향 = a2 = torch.tensor([2.0,0.1]).reshape(2,1)
정숙성향 = a3 = torch.tensor([2.5,1.0]).reshape(2,1)
영숙성향 = a4 = torch.tensor([0.1,1.9]).reshape(2,1)
순자성향 = a5 = torch.tensor([0.2,2.1]).reshape(2,1)
현숙성향 = a6 = torch.tensor([0.0,1.5]).reshape(2,1)
A = torch.concat([a1,a2,a3,a4,a5,a6],axis=1)
A # 각 column은 여성출연자들의 성향을 의미함 

tensor([[1.9000, 2.0000, 2.5000, 0.1000, 0.2000, 0.0000],
        [0.0000, 0.1000, 1.0000, 1.9000, 2.1000, 1.5000]])

In [32]:
영식성향 = b1 = torch.tensor([2.0,0.1]).reshape(2,1)
영철성향 = b2 = torch.tensor([1.9,0.2]).reshape(2,1)
영호성향 = b3 = torch.tensor([1.8,0.3]).reshape(2,1)
광수성향 = b4 = torch.tensor([0.3,2.1]).reshape(2,1)
상철성향 = b5 = torch.tensor([0.2,2.0]).reshape(2,1)
영수성향 = b6 = torch.tensor([0.1,1.9]).reshape(2,1)
B = torch.concat([b1,b2,b3,b4,b5,b6],axis=1)
B # 각 column은 남성출연자의 성향을 의미함

tensor([[2.0000, 1.9000, 1.8000, 0.3000, 0.2000, 0.1000],
        [0.1000, 0.2000, 0.3000, 2.1000, 2.0000, 1.9000]])

`-` 아래의 행렬곱 관찰 

In [33]:
A.T@B 

tensor([[3.8000, 3.6100, 3.4200, 0.5700, 0.3800, 0.1900],
        [4.0100, 3.8200, 3.6300, 0.8100, 0.6000, 0.3900],
        [5.1000, 4.9500, 4.8000, 2.8500, 2.5000, 2.1500],
        [0.3900, 0.5700, 0.7500, 4.0200, 3.8200, 3.6200],
        [0.6100, 0.8000, 0.9900, 4.4700, 4.2400, 4.0100],
        [0.1500, 0.3000, 0.4500, 3.1500, 3.0000, 2.8500]])

---저거 따져보자--- 

${\bf A}_{2 \times 6} = \begin{bmatrix} {\boldsymbol a}_{\text{옥순}} & {\boldsymbol a}_{\text{영자}}  & {\boldsymbol a}_{\text{정숙}} & {\boldsymbol a}_{\text{영숙}}  & {\boldsymbol a}_{\text{순자}}  & {\boldsymbol a}_{\text{현숙}} \end{bmatrix} = \begin{bmatrix} 1.9 & 2.0 & 2.5 & 0.1 & 0.2  & 0 \\  0 & 0.1 & 1.0 & 1.9 & 2.1  & 1.5  \end{bmatrix}$

${\bf B}_{2 \times 6} = \begin{bmatrix} {\boldsymbol b}_{\text{영식}} & {\boldsymbol b}_{\text{영철}}  & {\boldsymbol b}_{\text{영호}} & {\boldsymbol b}_{\text{광수}}  & {\boldsymbol b}_{\text{상철}}  & {\boldsymbol b}_{\text{영수}} \end{bmatrix} = \begin{bmatrix} 2.0 & 1.0 & 1.8 & 0.3 & 0.2 & 0.1 \\  0.1 & 0.2 & 0.3 & 2.1 & 2.0 & 1.9  \end{bmatrix}$

$\begin{align*}{\bf A}^\top @ {\bf B} & = \begin{bmatrix}
1.9 & 0.0 \\
2.0 & 0.1 \\
2.5 & 1.0 \\
0.1 & 1.9 \\
0.2 & 2.1 \\
0.0 & 1.5 \\
\end{bmatrix} \begin{bmatrix} 2.0 & 1.0 & 1.8 & 0.3 & 0.2 & 0.1 \\  0.1 & 0.2 & 0.3 & 2.1 & 2.0 & 1.9  \end{bmatrix} \\ \\ 
& = \begin{bmatrix}
3.8 & 3.61 & 3.42 & 0.57 & 0.38 & 0.19 \\
4.01 & 3.82 & 3.63 & 0.81 & 0.6 & 0.39 \\
5.1 & 4.95 & 4.8 & 2.85 & 2.5 & 2.15 \\
0.39 & 0.57 & 0.75 & 4.02 & 3.82 & 3.62 \\
0.61 & 0.8 & 0.99 & 4.47 & 4.24 & 4.01 \\
0.15 & 0.3 & 0.45 & 3.15 & 3 & 2.85 \\
\end{bmatrix} \\ \\ 
& =\begin{bmatrix}
{\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{옥순}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\
{\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{영자}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
{\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{정숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}} \\
{\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{영숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
{\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{순자}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
{\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영식}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영철}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영호}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{광수}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{상철}} & {\boldsymbol a}_{\text{현숙}}^\top @ {\boldsymbol b}^{ }_{\text{영수}}  \\
\end{bmatrix}\end{align*}$

`-` 궁합매트릭스: ${\bf A}^\top @ {\bf B}$를 계산하면 (6,6) 인 행렬이 나올텐데 이 행렬의 $(i,j)$의 원소는 $i$-th 여성출연자와 $j$-th 남성출연자가 얼마나 잘 맞는지를 나타내는 숫자가 된다. (숫자가 높을수록 잘 맞음) 이 수업에서는 이 매트릭스를 "궁합매트릭스" 라고 정의하자. 

- 걱정1: 5.0이 넘는 값도 있네?
- 걱정2: 잘못잡으면 음수가 나올지도? 
- 로지스틱 느낌 나지 않음??

`-` 주어진 자료와 우리가 임의로 만든 궁합매트릭스를 비교해보자. 

In [34]:
A.T @ B, torch.tensor(np.array(df_view)).float()

(tensor([[3.8000, 3.6100, 3.4200, 0.5700, 0.3800, 0.1900],
         [4.0100, 3.8200, 3.6300, 0.8100, 0.6000, 0.3900],
         [5.1000, 4.9500, 4.8000, 2.8500, 2.5000, 2.1500],
         [0.3900, 0.5700, 0.7500, 4.0200, 3.8200, 3.6200],
         [0.6100, 0.8000, 0.9900, 4.4700, 4.2400, 4.0100],
         [0.1500, 0.3000, 0.4500, 3.1500, 3.0000, 2.8500]]),
 tensor([[4.5000, 4.0000, 3.2000,    nan, 2.1000,    nan, 1.3000, 1.2000],
         [   nan, 4.3000, 3.5000, 3.4000, 2.3000, 2.6000,    nan, 1.8000],
         [3.8000,    nan, 4.3000, 4.6000, 1.5000, 1.0000,    nan, 2.5000],
         [3.9000, 3.1000,    nan, 4.1000, 1.2000,    nan, 2.1000, 2.3000],
         [   nan, 2.9000, 1.2000, 1.9000,    nan, 4.5000, 3.9000, 3.2000],
         [2.1000, 2.4000,    nan, 1.5000, 4.5000, 4.5000, 3.9000,    nan],
         [1.8000, 1.0000, 2.3000,    nan,    nan, 3.9000, 4.9000, 4.8000],
         [   nan, 1.9000, 2.2000, 3.1000, 3.7000, 4.2000, 4.2000,    nan],
         [4.9000, 4.7000, 4.9000, 4.9000, 4.

`-` 우리의 전략

1. ${\bf A}^\top @~ {\bf B}$의 값과 `df_view` 의 값이 `nan`을 제외한 곳에서 거의 비슷하게 되도록 ${\bf A}$, ${\bf B}$를 잘 때려맞추면 되는것 아니야?

2. 1을 만족하는 ${\bf A}$, ${\bf B}$를 찾았으면 그 숫자들을 이용하여 `df_view`의 `nan` 을 추정한다. 

`-` 따라서 모형은 아래와 같이 볼 수 있다. 

$${\tt df\_view} \approx {\bf A}^\top @~ {\bf B}$$

`-` 아래의 정보를 참고하여 위의 수식을 다시 정리하면..

In [35]:
df_view

,영식(IN),영철(IN),영호(IS),광수(IS),상철(EN),영수(EN),규빈(ES),다호(ES)
옥순(IN),4.5,4.0,3.2,NaN,2.1,NaN,1.3,1.2
영자(IN),NaN,4.3,3.5,3.4,2.3,2.6,NaN,1.8
정숙(IS),3.8,NaN,4.3,4.6,1.5,1.0,NaN,2.5
영숙(IS),3.9,3.1,NaN,4.1,1.2,NaN,2.1,2.3
순자(EN),NaN,2.9,1.2,1.9,NaN,4.5,3.9,3.2
현숙(EN),2.1,2.4,NaN,1.5,4.5,4.5,3.9,NaN
서연(ES),1.8,1.0,2.3,NaN,NaN,3.9,4.9,4.8
보람(ES),NaN,1.9,2.2,3.1,3.7,4.2,4.2,NaN
하니(I),4.9,4.7,4.9,4.9,4.2,4.1,4.3,3.9


In [36]:
display(pd.DataFrame(A.T.tolist()).set_axis(['옥순(IN)','영자(IS)','정숙(I?)','영숙(EN)','순자(ES)','현숙(E?)']).set_axis(['IE성향','NS성향'],axis=1))
display(pd.DataFrame(B.T.tolist()).set_axis(['영식(IN)','영철(IS)','영호(IN)','광수(ES)','상철(EN)','영수(ES)']).set_axis(['IE성향','NS성향'],axis=1))

,IE성향,NS성향
옥순(IN),1.9,0.0
영자(IS),2.0,0.1
정숙(I?),2.5,1.0
영숙(EN),0.1,1.9
순자(ES),0.2,2.1
현숙(E?),0.0,1.5


,IE성향,NS성향
영식(IN),2.0,0.1
영철(IS),1.9,0.2
영호(IN),1.8,0.3
광수(ES),0.3,2.1
상철(EN),0.2,2.0
영수(ES),0.1,1.9


---이런식으로?---

|W|M|W_MBTI := ${\boldsymbol a}_{W}^T$|M_MBTI := ${\boldsymbol b}_{M}^T$|yhat|y| 
|:-:|:-:|:-:|:-:|:-:|:-:
|옥순|영식|[1.9, 0.0] | [2.0, 0.1] |3.8| 3.9|
|옥순|영철|[1.9, 0.0] | [1.9, 0.2] |3.6| 4.1|
|옥순|광수|[1.9, 0.0] | [0.3, 2.1] |0.57| 0.5|
|옥순|상철|[1.9, 0.0] | [0.2, 2.0] |0.38| 0.3|
|...|...|... | ... | ...|...|
|현숙|영철|[0.0, 1.5] | [1.9, 0.2] |0.3| 0.2|
|현숙|영호|[0.0, 1.5] | [1.8, 0.3] |0.45| 0.2|
|현숙|상철|[0.0, 1.5] | [0.2, 2.0] |3.0| 0.2|
|현숙|영수|[0.0, 1.5] | [0.1, 1.9] |2.85| 0.2|

`-` 이렇게 정리하면 `yhat` $\approx$ `y` 를 만족하도록 하면 된다. 

- `yhat`은 어떻게 구하지? `(W_MBTI * M_MBTI).sum()`? 
- 그럼 `W_IE`,`M_IE`는 어떻게 구하지?? 생각해보니까 데이터에서 주어진건 아니잖아

`-` `W_IE`,`M_IE` 를 어떻게 만들지?

- 그전엔 어떻게 했지?? `W`을 보고 적당히 특징을 상상하고 `W_IE`를 때려넣음 + `M`를 보고 적당히 특징을 상상하고 `M_IE`를 넣음. 
- 자동화하려면? `W` $\to$ `W_IE` 인 함수를 만들고 `M` $\to$ `M_IE` 인 함수를 만들자.

`-` 앞으로 할일1:  `W` $\to$ `W_IE` 인 함수를 만들고 `M` $\to$ `M_IE` 인 함수를 만들자. 즉 
    
- 옥순 $\to$ 1.9, 0.0
- 영자 $\to$ 2.0, 0.1 
...
- 현숙 $\to$ 0.0, 1.5 

로 가는 함수를 하나 구현하고 

- 영식 $\to$ 2.0, 0.1
- 영철 $\to$ 1.9, 0.2 
...
- 영수 $\to$ 0.1, 1.9 

를 와 같은 함수도 하나 구현한다. 

`-` 앞으로 할일2: 우리가 익숙한 셋팅 (step1~4)
    
1. `W_IE`, `M_IE` $\to$ `yhat` 를 수행
2. `y` $\approx$ `yhat` 인지 체크: `loss = loss_fn(yhat,y)`
3. `loss.backward()`
4. 더 나은 `W_IE`, `M_IE` 가 생기도록 update 

## C. 할일1: `W` $\to$ `W_IE`, `M` $\to$ `M_IE` 의 구현

`-` dataframe의 변형 

In [37]:
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
df[:5]

,W,M,y
0,옥순(IN),영식(IN),4.5
1,옥순(IN),영철(IN),4.0
2,옥순(IN),영호(IS),3.2
3,옥순(IN),상철(EN),2.1
4,옥순(IN),규빈(ES),1.3


`-` 이름을 숫자화

In [38]:
w = {'옥순(IN)':0, '영자(IS)':1, '정숙(I?)':2, '영숙(EN)':3, '순자(ES)':4, '현숙(E?)':5}
m = {'영식(IN)':0, '영철(IS)':1, '영호(IN)':2, '광수(ES)':3, '상철(EN)':4, '영수(ES)':5}

In [39]:
df['X1'] = df['W'].map(w)
df['X2'] = df['M'].map(m)
df[:5]

,W,M,y,X1,X2
0,옥순(IN),영식(IN),4.5,0.0,0.0
1,옥순(IN),영철(IN),4.0,0.0,NaN
2,옥순(IN),영호(IS),3.2,0.0,NaN
3,옥순(IN),상철(EN),2.1,0.0,4.0
4,옥순(IN),규빈(ES),1.3,0.0,NaN


`-` 텐서화 + one_hot-인코딩 

In [40]:
y = torch.tensor(df['y']).float().reshape(-1,1)
E1 = torch.nn.functional.one_hot(torch.tensor(df['X1'])).float()
E2 = torch.nn.functional.one_hot(torch.tensor(df['X2'])).float()
print(f'y.shape: {y.shape}')
print(f'E1.shape: {E1.shape} // 이때 {E1.shape[-1]}은 여성참가자의 수')
print(f'E2.shape: {E2.shape} // 이때 {E2.shape[-1]}은 남성참가자의 수')

RuntimeError: one_hot is only applicable to index tensor.

`-` `X1` -> `X1_IE`, `X2` -> `X2_IE` 인 변환구현  

In [41]:
torch.manual_seed(43052)
l1 = torch.nn.Linear(in_features=6, out_features=2)
l2 = torch.nn.Linear(in_features=6, out_features=2)
W_IE = l1(E1) # W_IE = l1(onehot(X1))
M_IE = l2(E2) # M_IE = l2(onehot(X2))

NameError: name 'E1' is not defined

In [42]:
W_IE[::5], M_IE[::5]

NameError: name 'W_IE' is not defined

## D. 할일2: step1~4 수행 

`-` step1: yhat을 구하자. 

In [43]:
W_IE[::5] # 여성참가자의 성향들 

NameError: name 'W_IE' is not defined

In [44]:
M_IE[::5] # 남성참가자의 성향들

NameError: name 'M_IE' is not defined

In [45]:
(-0.4139 * 0.5298) + (0.2151 * 0.1328) # 궁합을 의미하는 값 

-0.19071894

In [46]:
(W_IE * M_IE).sum(axis=1).reshape(-1,1)[::5]

NameError: name 'W_IE' is not defined

출력결과를 0~5사이로 맞춰주자.

In [47]:
sig = torch.nn.Sigmoid()
yhat = sig((W_IE * M_IE).sum(axis=1).reshape(-1,1))*5
yhat

NameError: name 'W_IE' is not defined

`-` step2: 손실계산 

In [48]:
loss_fn = torch.nn.MSELoss()
loss = loss_fn(yhat,y)

NameError: name 'yhat' is not defined

`-` step3: 미분 

In [49]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1415, -0.3458,  0.1472,  0.3811,  0.1663, -0.3627],
         [ 0.1330, -0.2537, -0.1058,  0.2549,  0.3958, -0.2112]]),
 tensor([-0.2724,  0.0821]),
 tensor([[ 0.1881, -0.0676,  0.3482, -0.1746, -0.0257,  0.3702],
         [-0.0318,  0.1532, -0.3444,  0.2514,  0.1382, -0.2983]]),
 tensor([0.3417, 0.1646]))

In [50]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(None, None, None, None)

In [51]:
loss.backward()

NameError: name 'loss' is not defined

In [52]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1415, -0.3458,  0.1472,  0.3811,  0.1663, -0.3627],
         [ 0.1330, -0.2537, -0.1058,  0.2549,  0.3958, -0.2112]]),
 tensor([-0.2724,  0.0821]),
 tensor([[ 0.1881, -0.0676,  0.3482, -0.1746, -0.0257,  0.3702],
         [-0.0318,  0.1532, -0.3444,  0.2514,  0.1382, -0.2983]]),
 tensor([0.3417, 0.1646]))

In [53]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(None, None, None, None)

`-` step4: update 

In [54]:
optimizr = torch.optim.Adam(list(l1.parameters())+list(l2.parameters()))
optimizr.step()

In [55]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1415, -0.3458,  0.1472,  0.3811,  0.1663, -0.3627],
         [ 0.1330, -0.2537, -0.1058,  0.2549,  0.3958, -0.2112]]),
 tensor([-0.2724,  0.0821]),
 tensor([[ 0.1881, -0.0676,  0.3482, -0.1746, -0.0257,  0.3702],
         [-0.0318,  0.1532, -0.3444,  0.2514,  0.1382, -0.2983]]),
 tensor([0.3417, 0.1646]))

In [56]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(None, None, None, None)

In [57]:
optimizr.zero_grad()

In [58]:
l1.weight.data,l1.bias.data,l2.weight.data,l2.bias.data

(tensor([[-0.1415, -0.3458,  0.1472,  0.3811,  0.1663, -0.3627],
         [ 0.1330, -0.2537, -0.1058,  0.2549,  0.3958, -0.2112]]),
 tensor([-0.2724,  0.0821]),
 tensor([[ 0.1881, -0.0676,  0.3482, -0.1746, -0.0257,  0.3702],
         [-0.0318,  0.1532, -0.3444,  0.2514,  0.1382, -0.2983]]),
 tensor([0.3417, 0.1646]))

In [59]:
l1.weight.grad,l1.bias.grad,l2.weight.grad,l2.bias.grad 

(None, None, None, None)

# 5. 추천시스템의 학습

## A. Bias를 고려하지 않은 구현

`-` 데이터를 불러오기 

In [384]:
pd.read_csv('solo1.csv',index_col=0)

,영식(IN),영철(IN),영호(IS),광수(IS),상철(EN),영수(EN),규빈(ES),다호(ES)
옥순(IN),NaN,4.02,3.45,3.42,0.84,1.12,0.43,0.49
영자(IN),3.93,3.99,3.63,3.43,0.98,0.96,0.52,NaN
정숙(IS),3.52,3.42,4.05,4.06,0.39,NaN,0.93,0.99
영숙(IS),3.43,3.57,NaN,3.95,0.56,0.52,0.89,0.89
순자(EN),1.12,NaN,0.59,0.43,4.01,4.16,3.52,3.38
현숙(EN),0.94,1.05,0.32,0.45,4.02,3.78,NaN,3.54
서연(ES),0.51,0.56,0.88,0.89,3.50,3.64,4.04,4.10
보람(ES),0.48,0.51,1.03,NaN,3.52,4.00,3.82,NaN
하니(I),4.85,4.82,4.75,4.98,4.53,4.39,4.45,4.52


In [385]:
#df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/STML2022/main/posts/V.%20RecSys/2022-12-21-rcmdsolo.csv',index_col=0)
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
w = {'옥순(IN)':0, '영자(IN)':1, '정숙(IS)':2, '영숙(IS)':3, '순자(EN)':4, '현숙(EN)':5, '서연(ES)':6, '보람(ES)':7, '하니(I)':8}
m = {'영식(IN)':0, '영철(IN)':1, '영호(IS)':2, '광수(IS)':3, '상철(EN)':4, '영수(EN)':5, '규빈(ES)':6, '다호(ES)':7}
X1 = torch.tensor(df['W'].map(w)) # length-n vector 
X2 = torch.tensor(df['M'].map(m)) # length-n vector 
E1 = torch.nn.functional.one_hot(X1).float()
E2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df['y']).float().reshape(-1,1)

`-` 데이터의 차원 및 dtype가 헷갈릴수 있는데 아래와 같이 되어있어야한다. 

In [386]:
print(f"X1: shape = {X1.shape}, dtype = {X1.dtype}") # one-hot 인코딩 넣을거라서.. lenght-n 벡터로
print(f"X2: shape = {X2.shape}, dtype = {X2.dtype}") # one-hot 인코딩 넣을거라서.. lenght-n 벡터로
print(f"E1: shape = {E1.shape}, dtype = {E1.dtype}") # one-hot 인코딩 처리된 X1
print(f"E2: shape = {E2.shape}, dtype = {E2.dtype}") # one-hot 인코딩 처리된 X2
print(f"y: shape = {y.shape}, dtype = {y.dtype}")

X1: shape = torch.Size([64]), dtype = torch.int64
X2: shape = torch.Size([64]), dtype = torch.int64
E1: shape = torch.Size([64, 9]), dtype = torch.float32
E2: shape = torch.Size([64, 8]), dtype = torch.float32
y: shape = torch.Size([64, 1]), dtype = torch.float32


`-` 학습

In [424]:
# 
l1 = torch.nn.Linear(in_features=9,out_features=2)
l2 = torch.nn.Linear(in_features=8,out_features=2)
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(
    list(l1.parameters())+list(l2.parameters()),
    lr = 0.1
)
#---#
#torch.manual_seed(0)
for epoc in range(3000):
    ## step1 
    W_features = l1(E1) # W_features = l1(E1)
    M_features = l2(E2) # M_features = l1(E1)
    score = (W_features * M_features).sum(axis=1)
    yhat = sig(score).reshape(-1,1)*5
    ## step2 
    loss = loss_fn(yhat,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

`-` 학습된결과 

In [425]:
df_ebdspace = pd.DataFrame(torch.concat([W_features.data, M_features.data, W_features.data*M_features.data],axis=1))
df_ebdspace.columns = ['여성잠재특징1','여성잠재특징2','남성잠재특징1','남성잠재특징2','여성잠재특징1*남성잠재특징1','여성잠재특징2*남성잠재특징2']
df_yhat = pd.DataFrame(yhat.data,columns=['yhat'])

In [426]:
df_result = pd.concat([df,df_yhat,df_ebdspace],axis=1)
df_result[:56]

,W,M,y,yhat,여성잠재특징1,여성잠재특징2,남성잠재특징1,남성잠재특징2,여성잠재특징1*남성잠재특징1,여성잠재특징2*남성잠재특징2
0,옥순(IN),영철(IN),4.02,3.656063,0.392200,1.187522,1.839022,0.235381,0.721263,0.279520
1,옥순(IN),영호(IS),3.45,3.600563,0.392200,1.187522,1.948561,0.152245,0.764225,0.180795
2,옥순(IN),광수(IS),3.42,3.539338,0.392200,1.187522,2.556992,-0.099199,1.002851,-0.117801
3,옥순(IN),상철(EN),0.84,0.657057,0.392200,1.187522,0.444652,-1.737171,0.174392,-2.062929
4,옥순(IN),영수(EN),1.12,0.719664,0.392200,1.187522,0.199157,-1.567224,0.078109,-1.861112
5,옥순(IN),규빈(ES),0.43,0.659875,0.392200,1.187522,0.360230,-1.705140,0.141282,-2.024891
6,옥순(IN),다호(ES),0.49,0.750057,0.392200,1.187522,0.491759,-1.623026,0.192868,-1.927379
7,영자(IN),영식(IN),3.93,3.612577,0.475784,1.086457,2.020494,-0.003999,0.961318,-0.004345
8,영자(IN),영철(IN),3.99,3.779848,0.475784,1.086457,1.839022,0.235381,0.874976,0.255732
9,영자(IN),영호(IS),3.63,3.744262,0.475784,1.086457,1.948561,0.152245,0.927094,0.165408


## B. Bias를 고려한 구현

In [427]:
df_view = pd.read_csv('solo1.csv',index_col=0)

In [428]:
df_view = pd.read_csv('solo1.csv',index_col=0)
df_view

,영식(IN),영철(IN),영호(IS),광수(IS),상철(EN),영수(EN),규빈(ES),다호(ES)
옥순(IN),NaN,4.02,3.45,3.42,0.84,1.12,0.43,0.49
영자(IN),3.93,3.99,3.63,3.43,0.98,0.96,0.52,NaN
정숙(IS),3.52,3.42,4.05,4.06,0.39,NaN,0.93,0.99
영숙(IS),3.43,3.57,NaN,3.95,0.56,0.52,0.89,0.89
순자(EN),1.12,NaN,0.59,0.43,4.01,4.16,3.52,3.38
현숙(EN),0.94,1.05,0.32,0.45,4.02,3.78,NaN,3.54
서연(ES),0.51,0.56,0.88,0.89,3.50,3.64,4.04,4.10
보람(ES),0.48,0.51,1.03,NaN,3.52,4.00,3.82,NaN
하니(I),4.85,4.82,4.75,4.98,4.53,4.39,4.45,4.52


In [440]:
#df_view = pd.read_csv('https://raw.githubusercontent.com/guebin/DL2024/main/posts/solo.csv',index_col=0)
df_view = pd.read_csv('solo1.csv',index_col=0)
df = df_view.stack().reset_index().set_axis(['W','M','y'],axis=1)
w = {'옥순(IN)':0, '영자(IN)':1, '정숙(IS)':2, '영숙(IS)':3, '순자(EN)':4, '현숙(EN)':5, '서연(ES)':6, '보람(ES)':7, '하니(I)':8}
m = {'영식(IN)':0, '영철(IN)':1, '영호(IS)':2, '광수(IS)':3, '상철(EN)':4, '영수(EN)':5, '규빈(ES)':6, '다호(ES)':7}
X1 = torch.tensor(df['W'].map(w)) # length-n vector 
X2 = torch.tensor(df['M'].map(m)) # length-n vector 
E1 = torch.nn.functional.one_hot(X1).float()
E2 = torch.nn.functional.one_hot(X2).float()
y = torch.tensor(df['y']).float().reshape(-1,1)
#--#
torch.manual_seed(0)
l1 = torch.nn.Linear(in_features=9,out_features=2,bias=False)
b1 = torch.nn.Linear(in_features=9,out_features=1,bias=False)
l2 = torch.nn.Linear(in_features=8,out_features=2,bias=False)
b2 = torch.nn.Linear(in_features=8,out_features=1,bias=False)
sig = torch.nn.Sigmoid()
loss_fn = torch.nn.MSELoss()
optimizr = torch.optim.Adam(
    list(l1.parameters())+list(l2.parameters())+list(b1.parameters())+list(b2.parameters()),
    lr=0.1
)
#--#
for epoc in range(100):
    ## step1 
    W_features = l1(E1) 
    W_bias = b1(E1)
    M_features = l2(E2) 
    M_bias = b2(E2)
    dot = (W_features * M_features + W_bias + M_bias).sum(axis=1)
    yhat = sig(dot).reshape(-1,1) * 5
    ## step2 
    loss = loss_fn(yhat,y)
    ## step3 
    loss.backward()
    ## step4 
    optimizr.step()
    optimizr.zero_grad()

In [441]:
df_view

,영식(IN),영철(IN),영호(IS),광수(IS),상철(EN),영수(EN),규빈(ES),다호(ES)
옥순(IN),NaN,4.02,3.45,3.42,0.84,1.12,0.43,0.49
영자(IN),3.93,3.99,3.63,3.43,0.98,0.96,0.52,NaN
정숙(IS),3.52,3.42,4.05,4.06,0.39,NaN,0.93,0.99
영숙(IS),3.43,3.57,NaN,3.95,0.56,0.52,0.89,0.89
순자(EN),1.12,NaN,0.59,0.43,4.01,4.16,3.52,3.38
현숙(EN),0.94,1.05,0.32,0.45,4.02,3.78,NaN,3.54
서연(ES),0.51,0.56,0.88,0.89,3.50,3.64,4.04,4.10
보람(ES),0.48,0.51,1.03,NaN,3.52,4.00,3.82,NaN
하니(I),4.85,4.82,4.75,4.98,4.53,4.39,4.45,4.52


In [444]:
df_ebdspace = pd.DataFrame(torch.concat([W_features.data*M_features.data,W_bias.data, M_bias.data],axis=1))
df_ebdspace.columns = ['잠재특징1궁합','잠재특징2궁합','여성바이어스','남성바이어스']
#df_ebdspace.columns = ['여성잠재특징','여성바이어스','남성잠재특징','남성바이어스']
df_yhat = pd.DataFrame(yhat.data,columns=['yhat'])
print(loss)
pd.concat([df,df_yhat,df_ebdspace],axis=1)[:57]

tensor(0.0080, grad_fn=<MseLossBackward0>)


,W,M,y,yhat,잠재특징1궁합,잠재특징2궁합,여성바이어스,남성바이어스
0,옥순(IN),영철(IN),4.02,4.060366,1.422030,0.214267,-0.228450,0.142070
1,옥순(IN),영호(IS),3.45,3.469652,1.697226,-0.492482,-0.228450,0.035357
2,옥순(IN),광수(IS),3.42,3.365253,1.691007,-0.487136,-0.228450,-0.012478
3,옥순(IN),상철(EN),0.84,0.920789,-1.465944,0.127106,-0.228450,0.153654
4,옥순(IN),영수(EN),1.12,0.959823,-1.416197,0.010162,-0.228450,0.212820
5,옥순(IN),규빈(ES),0.43,0.559174,-1.282929,-0.465865,-0.228450,0.066779
6,옥순(IN),다호(ES),0.49,0.543288,-1.310801,-0.540438,-0.228450,0.101805
7,영자(IN),영식(IN),3.93,3.983574,1.425729,0.158860,-0.229443,0.120092
8,영자(IN),영철(IN),3.99,3.996445,1.399013,0.157588,-0.229443,0.142070
9,영자(IN),영호(IS),3.63,3.574573,1.669755,-0.362209,-0.229443,0.035357


`-` 예측: bias가 좀 더 반영된 느낌 

In [348]:
# js_onehot = torch.tensor([[0., 0., 1., 0., 0., 0.]])
# ys_onehot = torch.tensor([[1., 0., 0., 0., 0., 0.]])
score = (l1(js_onehot) * l2(ys_onehot) + b1(js_onehot) + b2(ys_onehot)).sum(axis=1) 
sig(score)*10

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x6 and 9x2)

In [349]:
# hs_onehot = torch.tensor([[0., 0., 0., 0., 0., 1.]])
# ys_onehot = torch.tensor([[1., 0., 0., 0., 0., 0.]])
score = (l1(hs_onehot) * l2(ys_onehot) + b1(hs_onehot) + b2(ys_onehot)).sum(axis=1) 
sig(score)*10

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x6 and 9x2)

## C. 은닉층의 해석 